In [41]:
import pandas as pd
df = pd.read_csv('C:\\Users\\Mahaboob\\Downloads\\BX-CSV-Dump\\BX-Books.csv', engine='c', on_bad_lines='skip', encoding='latin-1', sep=';')
df.head()


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [43]:
df.duplicated(subset='Book-Title').sum()

29225

In [44]:
df = df.drop_duplicates(subset='Book-Title')

In [45]:
df.duplicated(subset='Book-Title').sum()

0

In [46]:
sample_size = 15000
df = df.sample(n=sample_size, replace=False, random_state=490)

df = df.reset_index()
df = df.drop('index',axis=1)

In [47]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0060972491,"Thank You, Jeeves",P.G. Wodehouse,1989,Perennial,http://images.amazon.com/images/P/0060972491.0...,http://images.amazon.com/images/P/0060972491.0...,http://images.amazon.com/images/P/0060972491.0...
1,015602781X,Pincher Martin: The Two Deaths of Christopher ...,William Golding,2002,Harvest Books,http://images.amazon.com/images/P/015602781X.0...,http://images.amazon.com/images/P/015602781X.0...,http://images.amazon.com/images/P/015602781X.0...
2,1561441627,The Nutcracker (Christmas Fun-to-Read Fairy Ta...,Roberta Collier-Morales,1992,Modern Publishing,http://images.amazon.com/images/P/1561441627.0...,http://images.amazon.com/images/P/1561441627.0...,http://images.amazon.com/images/P/1561441627.0...
3,1402708610,I Know a Rhino,Charles Fuge,2003,Sterling,http://images.amazon.com/images/P/1402708610.0...,http://images.amazon.com/images/P/1402708610.0...,http://images.amazon.com/images/P/1402708610.0...
4,0441586325,Norby and the Oldest Dragon,Janet Asimov,1993,Ace Books,http://images.amazon.com/images/P/0441586325.0...,http://images.amazon.com/images/P/0441586325.0...,http://images.amazon.com/images/P/0441586325.0...


In [48]:
def clean_text(author):
    result = str(author).lower()
    return(result.replace(' ',''))

In [49]:
df['Book-Author'] = df['Book-Author'].apply(clean_text)

In [50]:
df['Book-Title'] = df['Book-Title'].str.lower()
df['Publisher'] = df['Publisher'].str.lower()

In [51]:
df2 = df.drop(['ISBN','Image-URL-S','Image-URL-M','Image-URL-L','Year-Of-Publication'],axis=1)

df2['data'] = df2[df2.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

print(df2['data'].head())

0                     p.g.wodehouse perennial
1                williamgolding harvest books
2    robertacollier-morales modern publishing
3                        charlesfuge sterling
4                       janetasimov ace books
Name: data, dtype: object


In [52]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df2['data'])

In [53]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [54]:
df = pd.DataFrame(similarities, columns=df['Book-Title'], index=df['Book-Title']).reset_index()

df.head()

Book-Title,Book-Title,"thank you, jeeves",pincher martin: the two deaths of christopher martin,the nutcracker (christmas fun-to-read fairy tales ser),i know a rhino,norby and the oldest dragon,his eminence and hizzoner,el oro de los sueã±os (leer en espaã±ol nivel 4),celestine insights,feng shui and money: a nine-week program for creating wealth using ancient principles and techniques,...,"the santa suit (harlequin american romance, no. 708)",where the gods reign: plants and peoples of the colombian amazon,the terrible troll-bird,virginia embrace (magnolia road),"take 5, volume 4 (under the knife, adam's gift, return to yesterday, everything but time, marriage diamond style)",taking the anxiety out of taking tests: a step-by-step guide,encyclopedia of women's health issues,fodor's 2002 new york city (fodor's new york city),"c. s. lewis: a biography,revised edition",the late night muse: a novel
0,"thank you, jeeves",1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,pincher martin: the two deaths of christopher ...,0.0,1.000000,0.0,0.0,0.333333,0.288675,0.000000,0.288675,0.0,...,0.0,0.0,0.235702,0.288675,0.0,0.0,0.0,0.0,0.0,0.0
2,the nutcracker (christmas fun-to-read fairy ta...,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.223607,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,i know a rhino,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,norby and the oldest dragon,0.0,0.333333,0.0,0.0,1.000000,0.288675,0.000000,0.288675,0.0,...,0.0,0.0,0.235702,0.288675,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
input_book = 'far beyond the stars (star trek deep space nine)'
recommendations = pd.DataFrame(df.nlargest(11,input_book)['Book-Title'])
recommendations = recommendations[recommendations['Book-Title']!=input_book]
print(recommendations)

                                             Book-Title
768    to storm heaven (star trek: the next generation)
802   the star trek chronology : a history of the fu...
1472           gateways #1:  one small step (star trek)
2051           reunion (star trek: the next generation)
2891                marooned (star trek voyager, no 14)
3727     maelstrom (ultima: the technocrat war, book 3)
3991         belle terre (star trek: new earth, book 2)
4567  the way of the warrior (star trek deep space n...
4757  a rock and a hard place:star trek next generat...
5812               the joy machine (star trek, book 80)
